In [7]:
import requests
from bs4 import BeautifulSoup
import re
import json

url = "https://www.peilingennederland.nl/alle-peilingen.html"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

spans = soup.find_all("span", style=re.compile("color:rgb\(42, 42, 42\)"))
party_pattern = re.compile(r"([A-Za-z0-9\-\/]+):\s*(\d+)")
seat_distribution = {}

for span in spans:
    text = span.decode_contents()
    if party_pattern.search(text):
        for match in party_pattern.finditer(text):
            party = match.group(1).replace('\u200b', '').strip()
            seats = int(match.group(2))
            seat_distribution[party] = seats
        break
if 'GL-PvdA' in seat_distribution:
    seat_distribution['GL/PvdA'] = seat_distribution.pop('GL-PvdA')
    # Sort by seat count descending
    seat_distribution = dict(sorted(seat_distribution.items(), key=lambda x: -x[1]))

print("Seat Distribution:", seat_distribution)

with open("verdeling.json", "w", encoding="utf-8") as f:
    json.dump(seat_distribution, f, ensure_ascii=False, indent=2)


<>:10: SyntaxWarning: invalid escape sequence '\('
<>:10: SyntaxWarning: invalid escape sequence '\('
/tmp/ipykernel_55857/559699779.py:10: SyntaxWarning: invalid escape sequence '\('
  spans = soup.find_all("span", style=re.compile("color:rgb\(42, 42, 42\)"))


Seat Distribution: {'PVV': 29, 'GL/PvdA': 26, 'CDA': 22, 'VVD': 16, 'D66': 12, 'JA21': 12, 'SP': 7, 'FvD': 5, 'DENK': 4, 'BBB': 4, 'PvdD': 3, 'CU': 3, 'SGP': 3, 'Volt': 3, '50PLUS': 1, 'NSC': 0}


In [8]:
%run coalition-calculations-no-biggest-party.py

if __name__ == "__main__":
    kabinetten, zetels, ek_zetels, topic_vectors = load_data()
    coalition_counter = build_coalition_frequency(kabinetten)

#     # User Input  # <<—— Add the (alleged) seat distribution for the election you want to predict
#     seat_distribution = {
# }

    Jaar = 2025  # <<—— Add the election year for Eerste Kamer seat distribution

    predictions = predict_coalitions(
        seat_distribution, 
        coalition_counter,
        ek_zetels=ek_zetels, 
        Jaar=Jaar, 
        threshold=76, 
        top_k=7,
        topic_vectors=topic_vectors
    )


    for p in predictions:
        print(f"Coalition: {p['coalition']}")
        print(f"  Seats: {p['seats']}")
        print(f"  History Score: {p['historical_score']}")
        print(f"  Ideology Score: {p['ideology_score']}")
        print(f"  EK Score: {p['ek_score']}")  # Optional: show EK alignment
        print(f"  EK Seats: {p['ek_total_seats']}")
        print(f"  JSD Penalty: {p['jsd_penalty']}")
        print(f"  Party Penalty: {p['party_penalty']}")
        print(f"  Surplus Penalty: {p['surplus_penalty']}")
        print(f"  Final Score: {p['final_score']}%")
        print("")


Coalition: ('GL/PvdA', 'CDA', 'VVD', 'D66')
  Seats: 76
  History Score: 1.17
  Ideology Score: 2.25
  EK Score: 0.47
  EK Seats: 35
  JSD Penalty: 0.05
  Party Penalty: 0
  Surplus Penalty: 0.0
  Final Score: 17.3%

Coalition: ('GL/PvdA', 'CDA', 'VVD', 'JA21')
  Seats: 76
  History Score: 1.05
  Ideology Score: 2.83
  EK Score: 0.4
  EK Seats: 30
  JSD Penalty: 0.08
  Party Penalty: 0
  Surplus Penalty: 0.0
  Final Score: 0%

Coalition: ('GL/PvdA', 'CDA', 'D66', 'JA21', 'DENK')
  Seats: 76
  History Score: 0.83
  Ideology Score: 2.93
  EK Score: 0.33
  EK Seats: 25
  JSD Penalty: 0.1
  Party Penalty: 2
  Surplus Penalty: 0.0
  Final Score: 0%

Coalition: ('GL/PvdA', 'CDA', 'VVD', 'SP', 'DENK', '50PLUS')
  Seats: 76
  History Score: 1.05
  Ideology Score: 2.15
  EK Score: 0.45
  EK Seats: 34
  JSD Penalty: 0.11
  Party Penalty: 4
  Surplus Penalty: 0.0
  Final Score: 0%

Coalition: ('GL/PvdA', 'CDA', 'D66', 'JA21', 'PvdD', '50PLUS')
  Seats: 76
  History Score: 0.83
  Ideology Score: 3

In [9]:
import json

# Add seat_distribution to each coalition for the visualizer
for p in predictions:
    p['seat_distribution'] = {party: seat_distribution[party] for party in p['coalition']}
    p['coalition'] = list(p['coalition'])  # Convert tuple to list for JSON

# Only coalitions with >0% final score, else top 3
filtered = [p for p in predictions if p['final_score'] > 0]
if not filtered:
    filtered = sorted(predictions, key=lambda x: -x['final_score'])[:3]

with open('coalition_data_any.json', 'w', encoding='utf-8') as f:
    json.dump(filtered, f, ensure_ascii=False, indent=2)

In [10]:
%run coalition-calculations.py

if __name__ == "__main__":
    kabinetten, zetels, ek_zetels, topic_vectors = load_data()
    coalition_counter = build_coalition_frequency(kabinetten)

#     # User Input  # <<—— Add the (alleged) seat distribution for the election you want to predict
#     seat_distribution = {

# }

    Jaar = 2025  # <<—— Add the election year for Eerste Kamer seat distribution

    predictions = predict_coalitions(
        seat_distribution, 
        coalition_counter, 
        ek_zetels=ek_zetels, 
        Jaar=Jaar, 
        threshold=76, 
        top_k=7,
        topic_vectors=topic_vectors
    )


    for p in predictions:
        print(f"Coalition: {p['coalition']}")
        print(f"  Seats: {p['seats']}")
        print(f"  History Score: {p['historical_score']}")
        print(f"  Ideology Score: {p['ideology_score']}")
        print(f"  EK Score: {p['ek_score']}")  # Optional: show EK alignment
        print(f"  EK Seats: {p['ek_total_seats']}")
        print(f"  JSD Penalty: {p['jsd_penalty']}")
        print(f"  Party Penalty: {p['party_penalty']}")
        print(f"  Surplus Penalty: {p['surplus_penalty']}")
        print(f"  Final Score: {p['final_score']}%")
        print("")


In [11]:
import json

# Add seat_distribution to each coalition for the visualizer
for p in predictions:
    p['seat_distribution'] = {party: seat_distribution[party] for party in p['coalition']}
    p['coalition'] = list(p['coalition'])  # Convert tuple to list for JSON

# Only coalitions with >0% final score, else top 3
filtered = [p for p in predictions if p['final_score'] > 0]
if not filtered:
    filtered = sorted(predictions, key=lambda x: -x['final_score'])[:3]

with open('coalition_data_with_biggest.json', 'w', encoding='utf-8') as f:
    json.dump(filtered, f, ensure_ascii=False, indent=2)

In [12]:
# parties_2023 = {
        # 'VVD': 24,
        # 'GL/PvdA': 25,
        # 'D66': 9,
        # 'CDA': 5,
        # 'CU': 3,
        # 'PVV': 37,
        # 'BBB': 7,
        # 'SP': 5,
        # 'PvdD': 3,
        # 'NSC': 20,
        # 'DENK': 3,
        # 'FvD': 3,
        # 'SGP': 3,
        # 'Volt': 2,
        # 'JA21': 1
# }

# parties_2021 = {
        # 'PVV':17,
        # 'VVD':34,
        # 'D66':24,
        # 'BBB':1,
        # 'CDA':15,
        # 'SP':9,
        # 'FvD':8,
        # 'PvdD':6,
        # 'SGP':3,
        # 'ChristenUnie':5,
        # 'DENK':3,
        # 'Volt':3,
        # 'JA21':3,
        # 'PvdA':9,
        # 'GroenLinks':8,
        # '50Plus':1,
        # 'BIJ1':1,
# }

# peiling_27052025 = {
        # 'VVD': 30,
        # 'GL/PvdA': 28,
        # 'D66': 10,
        # 'CDA': 18,
        # 'CU': 3,
        # 'PVV': 28,
        # 'BBB': 2,
        # 'SP': 8,
        # 'PvdD': 5,
        # 'NSC': 2,
        # 'DENK': 4,
        # 'FvD': 3,
        # 'SGP': 3,
        # 'Volt': 3,
        # 'JA21': 3
# }

# peiling_10062025 = {
        # 'VVD': 26,
        # 'GL/PvdA': 25,
        # 'D66': 10,
        # 'CDA': 20,
        # 'CU': 5,
        # 'PVV': 33,
        # 'BBB': 3,
        # 'SP': 6,
        # 'PvdD': 5,
        # 'NSC': 1,
        # 'DENK': 4,
        # 'FvD': 3,
        # 'SGP': 3,
        # 'Volt': 4,
        # 'JA21': 2
# }